# Biblical Exegesis Agent

This agent is equipped with multiple API-endpoint tools for answering questions about the Bible. The agent itself is a plan and execute agent, which calls an OpenAI function agent who calls specific APIs. It is able to answer questions about the Bible by using the following API-endpoints:

## Primary source data
- ATLAS endpoint
	- [ATLAS API agent endpoint](https://ryderwishart--atlas-agent-answer-question-using-atlas.modal.run/?query=What%20are%20the%20discourse%20features%20of%20JHN%201:11?) ([notebook](https://github.com/ryderwishart/biblical-machine-learning/blob/main/gpt-inferences/macula-api-agent.ipynb))
- MACULA data
	- TODO: MACULA dataframe agent endpoint
	- [Syntax bracket endpoint](https://ryderwishart--syntax-agent-get-syntax-for-query.modal.run/?query=Jesus%20in%201%20John%201:1) 
	- [Syntax agent endpoint](https://ryderwishart--syntax-agent-syntax-qa-chain.modal.run/?query=What%20is%20the%20subject%20of%20the%20verb%20%27%20%27love%27%27%20in%20Matthew%205%3A44%3F)
- TODO: OpenText data

## Secondary Source Data
- [Tyndale Open Study Notes](https://drive.google.com/file/d/1OrrP7HOJJx4gU_WgHSfo9myTbIAnYFqP/view?usp=sharing) - several types of resource
	- [RetrievalQA chain endpoint](https://ryderwishart--tyndale-chroma-question.modal.run/?query=Where%20did%20Jesus%20heal%20the%20man%20born%20blind?)
	- [Return documents endpoint](https://ryderwishart--tyndale-chroma-get-documents.modal.run/?query=jesus%20speaks%20to%20john)

In [2]:
import getpass, os 
secret_key = getpass.getpass('Enter OpenAI secret key: ') 
os.environ['OPENAI_API_KEY'] = secret_key

## Init OpenAI Function Agent

In [3]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool, tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
import requests

In [4]:
atlas_endpoint = "https://ryderwishart--atlas-agent-answer-question-using-atlas.modal.run/"
atlas_endpoint_example = "https://ryderwishart--atlas-agent-answer-question-using-atlas.modal.run/?query=What%20are%20the%20discourse%20features%20of%20JHN%201:11?"

syntax_bracket_endpoint = "https://ryderwishart--syntax-agent-get-syntax-for-query.modal.run/"
syntax_bracket_endpoint_example = "https://ryderwishart--syntax-agent-get-syntax-for-query.modal.run/?query=Jesus%20in%201%20John%201:1"

syntax_agent_endpoint = "https://ryderwishart--syntax-agent-syntax-qa-chain.modal.run/"
syntax_agent_endpoint = "https://ryderwishart--syntax-agent-syntax-qa-chain.modal.run/?query=What%20is%20the%20subject%20of%20the%20verb%20%27%20%27love%27%27%20in%20Matthew%205%3A44%3F"

tyndale_agent_endpoint = "https://ryderwishart--tyndale-chroma-question.modal.run/"
tyndale_documents_endpoint = "https://ryderwishart--tyndale-chroma-get-documents.modal.run/?query=jesus%20speaks%20to%20john"

endpoints = {
    "atlas": {
        "endpoint": atlas_endpoint,
        "example": atlas_endpoint_example,
    },
    "syntax-bracket": {
        "endpoint": syntax_bracket_endpoint,
        "example": syntax_bracket_endpoint_example,
    },
    "syntax-agent": {
        "endpoint": syntax_agent_endpoint,
        "example": syntax_agent_endpoint,
    },
    "tyndale-agent": {
        "endpoint": tyndale_agent_endpoint,
        "example": tyndale_documents_endpoint,
    },
}

In [12]:
# @tool
# def query_endpoint(query: str, endpoint_name: str):
#     """Ask a question of an endpoint. Possible endpoints are:
#     - atlas
#     - syntax_bracket
#     - syntax_agent
#     - tyndale_agent
#     - tyndale_documents
#     """
#     url_encoded_query = query.replace(" ", "%20")
#     endpoint, example = endpoints[endpoint_name].values()
#     url = f"{endpoint}?query={url_encoded_query}"
    
#     response = requests.get(url)
#     return response.json()

# Let's make one function tool per endpoint
@tool
def query_atlas(query: str):
    """Ask a question of the atlas endpoint."""
    url_encoded_query = query.replace(" ", "%20")
    endpoint, example = endpoints["atlas"].values()
    url = f"{endpoint}?query={url_encoded_query}"
    
    try:
        response = requests.get(url)
        return response.json()
    except:
        return {"error": "There was an error with the request. Please reformat request or try another tool."}

@tool
def query_syntax_bracket(query: str):
    """Ask a question of the syntax-bracket endpoint."""
    url_encoded_query = query.replace(" ", "%20")
    endpoint, example = endpoints["syntax-bracket"].values()
    url = f"{endpoint}?query={url_encoded_query}"
    
    response = requests.get(url)
    return response.json()

@tool
def query_syntax_agent(query: str):
    """Ask a question of the syntax-agent endpoint."""
    url_encoded_query = query.replace(" ", "%20")
    endpoint, example = endpoints["syntax-agent"].values()
    url = f"{endpoint}?query={url_encoded_query}"
    
    response = requests.get(url)
    return response.json()

@tool
def query_tyndale_agent(query: str):
    """Ask a question of the tyndale-agent endpoint."""
    url_encoded_query = query.replace(" ", "%20")
    endpoint, example = endpoints["tyndale-agent"].values()
    url = f"{endpoint}?query={url_encoded_query}"
    
    response = requests.get(url)
    return response.json()

@tool
def query_tyndale_documents(query: str):
    """Ask a question of the tyndale-documents endpoint."""
    url_encoded_query = query.replace(" ", "%20")
    endpoint, example = endpoints["tyndale-agent"].values()
    url = f"{endpoint}?query={url_encoded_query}"
    
    response = requests.get(url)
    return response.json()

In [13]:
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

tools = [
    Tool(
        name = "Query Atlas",
        func=query_atlas.run,
        description="useful for when you need to answer questions about New Testament discourse analysis. You may include a Bible verse such as 'Matthew 5:44' or '1 John 3:1' in your query, or ask about a discourse feature."
    ),
    Tool(
        name = "Query Syntax Bracket",
        func=query_syntax_bracket.run,
        description="useful for when you need to view a syntax analysis of a given Bible verse. You must include a Bible verse in your query."
    ),
    Tool(
        name = "Query Syntax Agent",
        func=query_syntax_agent.run,
        description="useful for when you need to answer questions about New Testament syntax. You must include a Bible verse such as 'Matthew 5:44' or '1 John 3:1' in your query."
    ),
    Tool(
        name = "Query Tyndale Agent",
        func=query_tyndale_agent.run,
        description="useful for when you need to answer encyclopedic questions about Bible",
    ),
    Tool(
        name = "Query Tyndale Documents",
        func=query_tyndale_documents.run,
        description="useful for when you need to access encyclopedic documents related to a given query directly",
    ),
]

In [14]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [15]:
# Test the agent
agent.run("Who is Barnabas?")

Error in on_chain_start callback: 'name'


I need to find information about Barnabas.
Action: Query Tyndale Agent
Action Input: "Who is Barnabas?"
Observation: {'question': 'Who is Barnabas?', 'answer': 'There is no information provided about who Barnabas is.\nSOURCES:', 'sources': ''}
Thought:I couldn't find any information about Barnabas using the Tyndale Agent.
Thought: I should try using the Tyndale Documents instead.
Action: Query Tyndale Documents
Action Input: "Barnabas"
Observation: {'question': 'Barnabas', 'answer': 'I don\'t know the answer to the question about "Barnabas."\nSOURCES:', 'sources': ''}
Thought:I couldn't find any information about Barnabas using the Tyndale Documents either.
Thought: I should try using Query Atlas to see if there is any information about Barnabas in the New Testament discourse analysis.
Action: Query Atlas
Action Input: "Barnabas"
Observation: {'error': 'There was an error with the request. Please reformat request or try another tool.'}
Thought:I couldn't find any information about Barn

'Barnabas is not mentioned in the New Testament discourse analysis or in the Tyndale documents. However, there is a Barnabas mentioned in the New Testament, specifically in the book of Acts, who was a companion of the apostle Paul.'

## Init Plan and Execute Agent

In [16]:
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.agents.tools import Tool

In [17]:
model = ChatOpenAI(temperature=0)

In [18]:
planner = load_chat_planner(model)

In [20]:
executor = load_agent_executor(llm, tools, verbose=True)

In [22]:
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [23]:
# Test
agent.run("Who is Barnabas?")

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'


steps=[Step(value='Search for information about Barnabas.'), Step(value='Gather relevant details about Barnabas, such as his background, occupation, and any notable achievements.'), Step(value='Summarize the information in a concise and informative manner.'), Step(value="Given the above steps taken, respond to the user's original question. \n")]Action:
```
{
  "action": "Query Tyndale Agent",
  "action_input": "Barnabas"
}
```
Observation: {'question': 'Barnabas', 'answer': 'I don\'t know the answer to the question about "Barnabas."\nSOURCES:', 'sources': ''}
Thought:I apologize for the confusion. Let me try to provide you with some information about Barnabas. 

Action:
```
{
  "action": "Query Tyndale Documents",
  "action_input": "Barnabas"
}
```
Observation: {'question': 'Barnabas', 'answer': 'I don\'t know the answer to the question about "Barnabas."\nSOURCES:', 'sources': ''}
Thought:

Error in on_chain_start callback: 'name'


I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?

> Finished chain.
*****

Step: Search for information about Barnabas.

Response: I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?Action:
```
{
  "action": "Query Tyndale Agent",
  "action_input": "Barnabas"
}
```
Observation: {'question': 'Barnabas', 'answer': 'I don\'t know the answer to the question about "Barnabas."\nSOURCES:', 'sources': ''}
Thought:

Error in on_chain_start callback: 'name'


I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?

> Finished chain.
*****

Step: Gather relevant details about Barnabas, such as his background, occupation, and any notable achievements.

Response: I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?Action:
```
{
  "action": "Query Tyndale Agent",
  "action_input": "Barnabas"
}
```
Observation: {'question': 'Barnabas', 'answer': 'I don\'t know the answer to the question about "Barnabas."\nSOURCES:', 'sources': ''}
Thought:

Error in on_chain_start callback: 'name'


I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?

> Finished chain.
*****

Step: Summarize the information in a concise and informative manner.

Response: I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?Action:
```
{
  "action": "Query Tyndale Agent",
  "action_input": "Who was Barnabas in the Bible?"
}
```
Observation: {'question': 'Who was Barnabas in the Bible?', 'answer': 'There is no mention of Barnabas in the provided content.\nSOURCES:', 'sources': ''}
Thought:I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?

> Finished chain.
*****

Step: Given the above steps taken, respond to the user's original question. 


Response: I

'I apologize for the inconvenience. It seems that I am unable to find any specific information about Barnabas at the moment. Is there anything else I can assist you with?'